# Vertical Interpolation

The `aus400.vertical` module has tools for interpolating Aus400 output to different vertical levels.

The Unified Model runs with hybrid height levels - levels follow the orography at low levels, and are at fixed heights at high levels. The true altitude of each grid cell is available in `stream='fx', variable='height_rho'`.

`aus400.vertical.to_plev(da, levels)` and `aus400.vertical.to_height(da, levels)` will vertically interpolate 'da' to a set of fixed pressure or height levels respectively.

In [1]:
import sys
sys.path.append('..')
import aus400

## Levels of the source data

Data as published is on model levels. There are two level types - 'rho' levels and 'theta' levels, each offset half a grid spacing from the other.

The `level_height` coordinate in the data is what the height of that level would be if the orographic height was at zero. `sigma` is how much the orographic height influences the height at that level.

The true height of a grid cell is `level_height + sigma * orography`, for convenience this is available pre-calculated under the `fx` stream.

In [2]:
ta = aus400.load(resolution='d0198', stream='mdl', variable='air_temp', ensemble=0, time='20170327T0000')['air_temp']
ta

<xarray.DataArray 'air_temp' (ensemble: 1, time: 1, model_level_number: 90, latitude: 2120, longitude: 2600)>
dask.array<broadcast_to, shape=(1, 1, 90, 2120, 2600), dtype=float32, chunksize=(1, 1, 1, 500, 500), chunktype=numpy.ndarray>
Coordinates:
  * time                     (time) datetime64[ns] 2017-03-27T01:00:00
  * model_level_number       (model_level_number) int32 1 2 3 4 ... 87 88 89 90
  * latitude                 (latitude) float32 -48.788 -48.7682 ... -6.8317986
  * longitude                (longitude) float32 107.52 107.539795 ... 158.9802
    forecast_period          timedelta64[ns] ...
    forecast_reference_time  datetime64[ns] ...
    level_height             (model_level_number) float64 dask.array<chunksize=(1,), meta=np.ndarray>
    sigma                    (model_level_number) float64 dask.array<chunksize=(1,), meta=np.ndarray>
  * ensemble                 (ensemble) int64 0
Attributes:
    standard_name:           air_temperature
    units:                   K
    source:                  Data from Met Office Unified Model
    um_version:              11.4
    STASH:                   [ 1 16  4]
    um_stash_source:         m01s16i004
    long_name:               TEMPERATURE ON THETA LEVELS
    stash_code:              16004
    wgdos_packing_accuracy:  2^-3
    resolution:              d0198
    stream:                  mdl

## Interpolating



In [3]:
ta_plev = aus400.vertical.to_plev(ta, [1000])
ta_plev

<xarray.DataArray 'air_temp' (ensemble: 1, time: 1, latitude: 2120, longitude: 2600, pressure: 1)>
dask.array<transpose, shape=(1, 1, 2120, 2600, 1), dtype=float64, chunksize=(1, 1, 500, 500, 1), chunktype=numpy.ndarray>
Coordinates:
  * time                     (time) datetime64[ns] 2017-03-27T01:00:00
  * latitude                 (latitude) float32 -48.788 -48.7682 ... -6.8317986
  * longitude                (longitude) float32 107.52 107.539795 ... 158.9802
    forecast_period          timedelta64[ns] 19:00:00
    forecast_reference_time  datetime64[ns] 2017-03-26T06:00:00
  * ensemble                 (ensemble) int64 0
  * pressure                 (pressure) int64 1000

In [8]:
ta_plev.squeeze()[1500:2000, 1500:2000].plot.pcolormesh()

Error in callback <function flush_figures at 0x7f69886d6a60> (for post_execute):


KeyboardInterrupt: 